In [11]:

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    !pip install -q -U transformers
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"


# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


#COPIED MODULE PART

In [12]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
max_id = len(tokenizer.word_index) # 고유 글자 개수.
dataset_size = tokenizer.document_count # 총 글자 개수.

In [13]:
#상태를 가지고, 모델이 마지막 상태를 다음 훈련 배치의 초기상태로 사용한다는
#Stateful RNN
#각 입력 시퀀스가 이전 배치 시퀀스 끝난 시점 시작.
#순차적인 입력 시퀀스를 요함


In [18]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100


In [22]:
batch_size = 32 #길이가 동일한 32개 텍스트로 나누고 
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:#각 나눠진 텍스트에 대해 아래의 작업을 반복한다.
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)#각 파트의 데이터셋을 뽑아와 텐서로 변환.
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)#윈도우를 만들어 나눠둠
    dataset = dataset.flat_map(lambda window: window.batch(window_length))#다시 플랫-맵으로.
    datasets.append(dataset) #datasets에 위 과정을 거친 배치가 차곡차곡 쌓인다.
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))#연속적인 배치가 만들어짐
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [23]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,#이전 에폭의 상태를 기억...!
                     dropout=0.2, #recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),#상태가 있는 rnn은 배치 크기를 알아야한다(배치에 있는 입력 시퀀스의 상태를 보존해야 한다.)
                     #따라서 batch_input_shape 매개변수 지정.
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [26]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

#매 에폭마다 상태를 재설정하는 콜백함수 제작.
#모든 메소드를 알 수는 없다. 


In [27]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])
#배치당 샘플이 하나(윈도우가 하나)기에 에폭을 늘렸다고 한다.

Epoch 1/50
313/313 [==============================] - 37s 12ms/step - loss: 2.9056
Epoch 2/50
313/313 [==============================] - 4s 11ms/step - loss: 2.2891
Epoch 3/50
313/313 [==============================] - 4s 11ms/step - loss: 2.1380
Epoch 4/50
313/313 [==============================] - 4s 12ms/step - loss: 2.0546
Epoch 5/50
313/313 [==============================] - 4s 12ms/step - loss: 1.9976
Epoch 6/50
313/313 [==============================] - 3s 11ms/step - loss: 1.9574
Epoch 7/50
313/313 [==============================] - 4s 11ms/step - loss: 1.9249
Epoch 8/50
313/313 [==============================] - 4s 11ms/step - loss: 1.9035
Epoch 9/50
313/313 [==============================] - 4s 11ms/step - loss: 1.8853
Epoch 10/50
313/313 [==============================] - 4s 11ms/step - loss: 1.8665
Epoch 11/50
313/313 [==============================] - 4s 11ms/step - loss: 1.8539
Epoch 12/50
313/313 [==============================] - 4s 11ms/step - loss: 1.8437
Epoch 13/50


In [28]:
#이 모델을 훈련한 뒤 동일한 크기의 배치로만 예측을 만들 수 있기에, 
#이후 동일한 크기의 배치로만 예측이 만들어지기때문에, 이런 제약을 없애기 위해 동일한 구조의 
#상태가 없는 모델을 만들고 상태가 있는 모델의 가중치를 복사한다.
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [29]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [30]:

stateless_model.set_weights(model.get_weights())#가중치를 위쪽 모델에서 가져오는 것.
model = stateless_model
